In [1]:
import pandas as pd

Before we start to create the mechanisms behind caluculating the elo scores, we first have to find columns which will be of use to us! Let's load in the drivers and results files.

In [2]:
drivers_df = pd.read_csv("data/drivers.csv")
drivers_df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [3]:
drivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   driverId     857 non-null    int64 
 1   driverRef    857 non-null    object
 2   number       857 non-null    object
 3   code         857 non-null    object
 4   forename     857 non-null    object
 5   surname      857 non-null    object
 6   dob          857 non-null    object
 7   nationality  857 non-null    object
 8   url          857 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.4+ KB


Columns of interest: driverId, driverRef, surname

In [4]:
drivers_df = drivers_df[['driverId','driverRef','surname']]
drivers_df.head()

,driverId,driverRef,surname
0,1,hamilton,Hamilton
1,2,heidfeld,Heidfeld
2,3,rosberg,Rosberg
3,4,alonso,Alonso
4,5,kovalainen,Kovalainen


Noice. We've got the columns we need from the drivers dataset.

In [5]:
constructors_df = pd.read_csv('data/constructors.csv')
constructors_df.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [6]:
constructors_df = constructors_df[['constructorId','constructorRef']]
constructors_df.head()

,constructorId,constructorRef
0,1,mclaren
1,2,bmw_sauber
2,3,williams
3,4,renault
4,5,toro_rosso


In [7]:
results_df = pd.read_csv("data/results.csv")
results_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [8]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25840 entries, 0 to 25839
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25840 non-null  int64  
 1   raceId           25840 non-null  int64  
 2   driverId         25840 non-null  int64  
 3   constructorId    25840 non-null  int64  
 4   number           25840 non-null  object 
 5   grid             25840 non-null  int64  
 6   position         25840 non-null  object 
 7   positionText     25840 non-null  object 
 8   positionOrder    25840 non-null  int64  
 9   points           25840 non-null  float64
 10  laps             25840 non-null  int64  
 11  time             25840 non-null  object 
 12  milliseconds     25840 non-null  object 
 13  fastestLap       25840 non-null  object 
 14  rank             25840 non-null  object 
 15  fastestLapTime   25840 non-null  object 
 16  fastestLapSpeed  25840 non-null  object 
 17  statusId    

In [9]:
results_df = results_df[['resultId','raceId','driverId','constructorId','grid','position','positionText','positionOrder','rank']]
results_df.head()

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,rank
0,1,18,1,1,1,1,1,1,2
1,2,18,2,2,5,2,2,2,3
2,3,18,3,3,7,3,3,3,5
3,4,18,4,4,11,4,4,4,7
4,5,18,5,1,3,5,5,5,1


Awesome! Now we have all the columns that we need. All that is left to do is merge the two dataframes together into one dataset.

In [10]:
results_df = pd.merge(results_df, drivers_df, how='left', on='driverId')
results_df.head()

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,rank,driverRef,surname
0,1,18,1,1,1,1,1,1,2,hamilton,Hamilton
1,2,18,2,2,5,2,2,2,3,heidfeld,Heidfeld
2,3,18,3,3,7,3,3,3,5,rosberg,Rosberg
3,4,18,4,4,11,4,4,4,7,alonso,Alonso
4,5,18,5,1,3,5,5,5,1,kovalainen,Kovalainen


In [11]:
results_df = pd.merge(results_df, constructors_df, how='left', on='constructorId')
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25840 entries, 0 to 25839
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   resultId        25840 non-null  int64 
 1   raceId          25840 non-null  int64 
 2   driverId        25840 non-null  int64 
 3   constructorId   25840 non-null  int64 
 4   grid            25840 non-null  int64 
 5   position        25840 non-null  object
 6   positionText    25840 non-null  object
 7   positionOrder   25840 non-null  int64 
 8   rank            25840 non-null  object
 9   driverRef       25840 non-null  object
 10  surname         25840 non-null  object
 11  constructorRef  25840 non-null  object
dtypes: int64(6), object(6)
memory usage: 2.6+ MB


In [12]:
results_df.head(10)

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,rank,driverRef,surname,constructorRef
0,1,18,1,1,1,1,1,1,2,hamilton,Hamilton,mclaren
1,2,18,2,2,5,2,2,2,3,heidfeld,Heidfeld,bmw_sauber
2,3,18,3,3,7,3,3,3,5,rosberg,Rosberg,williams
3,4,18,4,4,11,4,4,4,7,alonso,Alonso,renault
4,5,18,5,1,3,5,5,5,1,kovalainen,Kovalainen,mclaren
5,6,18,6,3,13,6,6,6,14,nakajima,Nakajima,williams
6,7,18,7,5,17,7,7,7,12,bourdais,Bourdais,toro_rosso
7,8,18,8,6,15,8,8,8,4,raikkonen,Räikkönen,ferrari
8,9,18,9,2,2,\N,R,9,9,kubica,Kubica,bmw_sauber
9,10,18,10,7,18,\N,R,10,13,glock,Glock,toyota


In [13]:
results_df.tail(10)

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,rank,driverRef,surname,constructorRef
25830,25836,1096,852,213,11,11,11,11,7,tsunoda,Tsunoda,alphatauri
25831,25837,1096,855,51,15,12,12,12,8,zhou,Zhou,alfa
25832,25838,1096,848,3,19,13,13,13,13,albon,Albon,williams
25833,25839,1096,842,213,17,14,14,14,19,gasly,Gasly,alphatauri
25834,25840,1096,822,51,18,15,15,15,16,bottas,Bottas,alfa
25835,25841,1096,854,210,12,16,16,16,12,mick_schumacher,Schumacher,haas
25836,25842,1096,825,210,16,17,17,17,20,kevin_magnussen,Magnussen,haas
25837,25843,1096,1,131,5,18,18,18,11,hamilton,Hamilton,mercedes
25838,25844,1096,849,3,20,19,19,19,14,latifi,Latifi,williams
25839,25845,1096,4,214,10,\N,R,20,17,alonso,Alonso,alpine


In [26]:
results_df[(results_df['driverId'] == 4)]

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,rank,driverRef,surname,constructorRef
3,4,18,4,4,11,4,4,4,7,alonso,Alonso,renault
29,30,19,4,4,7,8,8,8,10,alonso,Alonso,renault
53,54,20,4,4,10,10,10,10,13,alonso,Alonso,renault
80,81,21,4,4,2,\N,R,15,9,alonso,Alonso,renault
93,94,22,4,4,7,6,6,6,6,alonso,Alonso,renault
...,...,...,...,...,...,...,...,...,...,...,...,...
25746,25752,1092,4,214,7,7,7,7,2,alonso,Alonso,alpine
25766,25772,1093,4,214,14,7,7,7,8,alonso,Alonso,alpine
25798,25804,1094,4,214,9,19,19,19,11,alonso,Alonso,alpine
25804,25810,1095,4,214,17,5,5,5,4,alonso,Alonso,alpine


In [25]:
results_df[(results_df['driverId'] != 842) & (results_df['constructorId'] == 5) & (results_df['raceId'] == 983)]

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,rank,driverRef,surname,constructorRef
23675,23680,983,832,5,14,\N,R,19,19,sainz,Sainz,toro_rosso


In [20]:
results_df['driverId'].value_counts().keys().tolist()

[4,
 8,
 22,
 1,
 18,
 30,
 20,
 13,
 119,
 15,
 14,
 815,
 817,
 21,
 17,
 105,
 110,
 77,
 137,
 3,
 117,
 55,
 822,
 95,
 2,
 807,
 154,
 172,
 23,
 289,
 182,
 35,
 84,
 65,
 57,
 123,
 163,
 832,
 830,
 118,
 102,
 49,
 44,
 187,
 224,
 56,
 199,
 197,
 158,
 825,
 223,
 207,
 356,
 177,
 16,
 94,
 238,
 175,
 71,
 840,
 235,
 178,
 112,
 200,
 222,
 826,
 5,
 304,
 341,
 839,
 63,
 160,
 173,
 347,
 842,
 50,
 37,
 278,
 844,
 140,
 360,
 346,
 328,
 64,
 9,
 122,
 828,
 79,
 813,
 221,
 10,
 31,
 231,
 11,
 176,
 156,
 88,
 364,
 306,
 427,
 232,
 92,
 846,
 152,
 847,
 24,
 233,
 131,
 127,
 138,
 99,
 230,
 145,
 155,
 78,
 133,
 90,
 250,
 475,
 373,
 202,
 170,
 109,
 81,
 25,
 203,
 206,
 501,
 87,
 100,
 129,
 841,
 358,
 849,
 243,
 814,
 821,
 280,
 848,
 808,
 212,
 579,
 69,
 818,
 554,
 67,
 404,
 181,
 219,
 386,
 166,
 345,
 403,
 305,
 32,
 456,
 262,
 114,
 39,
 578,
 103,
 157,
 327,
 811,
 418,
 27,
 153,
 854,
 70,
 852,
 385,
 104,
 239,
 252,
 838,
 91,
 136,